<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Importar Librerias</strong></h4>
</div>

In [1]:
import ray
from ray import serve
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel, ConfigDict
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError
from sklearn.datasets import load_iris
import requests

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Conecta a un cluster de Ray ya iniciado</strong></h4>
</div>

In [2]:
try:
    ray.init(address="ray://127.0.0.1:10001")
    print("✅ Conectado a Ray existente.")
except Exception as e:
    sys.exit(f"❌ No se pudo conectar a ray://127.0.0.1:10001 → {e}")

2025-08-10 19:53:22,532	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.


✅ Conectado a Ray existente.


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Configuracion del Modelo</strong></h4>
</div>

In [3]:
try:
    app = FastAPI(
        title = "Import Predictor API",
        description = "Import Pipeline online inference"
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code = 422,
            content = { "cod": 422, "msg": "; ".join(mensajes), "data": None, "errors": exc.errors(), },
        )
    
    class Item(BaseModel):

        model_config = ConfigDict(extra='forbid')
        
        sepal_length: float
        sepal_width:  float
        petal_length: float
        petal_width:  float

    @serve.deployment( num_replicas=1, ray_actor_options={"num_cpus": 1} )
    
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self, model_uri: str):
            mlflow.set_tracking_uri("http://127.0.0.1:8080/")
            try:
                self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: Item):
            try:
                #payload = item.dict()
                payload = item.model_dump()
                df = pd.DataFrame([payload])
                score = float(self.model.predict_proba(df)[0][1])
                return {"cod": 200, "msg": "Score calculated successfully.", "data": {"em_eventprobability": score}, "errors": None }
            except Exception as e:
                #logger.error(f"Error in import prediction: {e}")
                return {"cod": 500, "msg": "Error calculating score", "data": None, "errors": str(e)}
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Desplegar el Modelo</strong></h4>
</div>

In [4]:
MODEL_URI = "models:/iris-models/1"   # o "models:/iris-models/Production"
NAME = "iris-models-v1"
ROUTE_PREFIX = "/v1/iris-models/score"

graph = ModelMlFlow.bind(MODEL_URI)

try:
    #serve.delete(NAME)  # opcional: limpiar antes
    serve.run(graph, name=NAME, route_prefix=ROUTE_PREFIX)
    print(f"✅ Listo en http://127.0.0.1:8000{ROUTE_PREFIX}")
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex}")
    try:
        serve.delete(NAME)
        print(f"🧹 App '{NAME}' eliminada por fallo de deploy.")
    except Exception as cleanup_ex:
        print(f"⚠️ No se pudo eliminar '{NAME}': {cleanup_ex}")

INFO 2025-08-10 19:53:26,616 serve 28664 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
INFO 2025-08-10 19:53:30,688 serve 28664 -- Application 'iris-models-v1' is ready at http://127.0.0.1:8000/v1/iris-models/score.


✅ Listo en http://127.0.0.1:8000/v1/iris-models/score


In [5]:
ray.shutdown()